## Description:
这次实验依然是用的Criteo数据集， 只不过由于原来的数据量太大， 为了在单机上能够运行， 做了采样， 取了很少的一部分进行实验。数据集位于data/文件夹下， train.csv是训练集， test.csv是测试集。 这个笔记本我们是做数据的读入和预处理操作， 具体步骤如下：
1. 读入数据集， 并进行缺失值的填充， 这里为了简单一些， 直接类别特征填充“-1”， 数值特征填充0
2. 类别特征的编码， 用的LabelEncoder编码， 数值特征的归一化处理
3. 划分开训练集和验证集保存到prepeocessed/文件夹下

## 导入包和数据集

In [12]:
# import packages
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.model_selection import train_test_split

In [2]:
# import data
train_df = pd.read_csv('./data/train.csv')
test_df = pd.read_csv('./data/test.csv')

print(train_df.shape, test_df.shape)

(1599, 41) (400, 40)


In [4]:
# 进行数据合并
label = train_df['Label']
del train_df['Label']

data_df = pd.concat((train_df, test_df))

In [8]:
del data_df['Id']

data_df.columns

Index(['I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11',
       'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9',
       'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19',
       'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26'],
      dtype='object')

In [9]:
# 特征分开类别
sparse_feas = [col for col in data_df.columns if col[0] == 'C']
dense_feas = [col for col in data_df.columns if col[0] == 'I']

In [11]:
# 填充缺失值
data_df[sparse_feas] = data_df[sparse_feas].fillna('-1')
data_df[dense_feas] = data_df[dense_feas].fillna(0)

## 数据预处理

In [14]:
# 进行编码  类别特征编码
for feat in sparse_feas:
    le = LabelEncoder()
    data_df[feat] = le.fit_transform(data_df[feat])

In [15]:
# 数值特征归一化
mms = MinMaxScaler()
data_df[dense_feas] = mms.fit_transform(data_df[dense_feas])

In [19]:
# 分开测试集和训练集
train = data_df[:train_df.shape[0]]
test = data_df[train_df.shape[0]:]

train['Label'] = label

C:\Users\ZhongqiangWu\Anaconda3\envs\pytorch_gpu\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


## 划分数据集

In [21]:
train_set, val_set = train_test_split(train, test_size = 0.2, random_state=2020)

In [23]:
train_set['Label'].value_counts()

0    998
1    281
Name: Label, dtype: int64

In [24]:
val_set['Label'].value_counts()

0    267
1     53
Name: Label, dtype: int64

In [25]:
# 保存文件
train_set.reset_index(drop=True, inplace=True)
val_set.reset_index(drop=True, inplace=True)

train_set.to_csv('preprocessed_data/train_set.csv', index=0)
val_set.to_csv('preprocessed_data/val_set.csv', index=0)
test.to_csv('preprocessed_data/test.csv', index=0)